In [ ]:
# prompt: mount  drive

from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# prompt: import yolo model v8x and predcit and save= true

!pip install ultralytics

from ultralytics import YOLO

# Load a model
model = YOLO('/content/drive/MyDrive/AI/football_analysis/best.pt')  # lodfad a pretrained model (recommended for trainng)
%cd /content/drive/MyDrive/AI/football_analysis
# Predict with the model
result = model.predict('/content/drive/MyDrive/AI/football_analysis/game.mp4', save=True)  #


In [ ]:
import numpy as np
import cv2
from ultralytics import YOLO
from norfair import Tracker, Detection

# Load YOLO model
model = YOLO('/content/drive/MyDrive/AI/football_analysis/best.pt')

# Video source and sink
VIDEO_PATH = '/content/drive/MyDrive/AI/football_analysis/game.mp4'
TARGET_VIDEO_PATH = '/content/drive/MyDrive/AI/football_analysis/tracked_game.mp4'

# Open video source
video_capture = cv2.VideoCapture(VIDEO_PATH)
fps = video_capture.get(cv2.CAP_PROP_FPS)
width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Open video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_writer = cv2.VideoWriter(TARGET_VIDEO_PATH, fourcc, fps, (width, height))

# Tracker
tracker = Tracker(
    distance_function='euclidean',
    distance_threshold=30
)

# Define color (red in BGR format)
color = (0, 0, 255)

# Process each frame
while True:
    ret, frame = video_capture.read()
    if not ret:
        break

    result = model(frame, agnostic_nms=True)[0]

    # Create detections
    detection_points = result.boxes.xyxy.cpu().numpy()
    detection_scores = result.boxes.conf.cpu().numpy()

    # Ensure scores are in the correct format
    if detection_scores.ndim == 1:
        detection_scores = np.expand_dims(detection_scores, axis=1)
    elif detection_scores.ndim != 2 or detection_scores.shape[1] != 1:
        raise ValueError("Unexpected shape of detection_scores")

    # Create norfair detections
    norfair_detections = []
    for i in range(len(detection_points)):
        points = detection_points[i].reshape((1, -1))  # Reshape to match (1, 4)
        score = detection_scores[i]
        norfair_detection = Detection(points=points, scores=score)
        norfair_detections.append(norfair_detection)

    # Convert goalkeeper to player
    detection_classes = result.boxes.cls.cpu().numpy().astype(int)
    for i, detection in enumerate(norfair_detections):
        if detection_classes[i] == 0:
            detection_classes[i] = 1

    # Track objects
    try:
        tracked_objects = tracker.update(detections=norfair_detections)
    except Exception as e:
        print("Error during tracking:", e)
        continue

    for obj in tracked_objects:
        # Get box coordinates
        x1, y1, x2, y2 = obj.estimate.flatten()

        # Calculate ellipse position and size
        center_x = int((x1 + x2) / 2)
        center_y = int(y2)
        width = int((x2 - x1) * 1.2)  # Slightly wider ellipses
        height = int((y2 - y1) / 4)    # Keep height the same

        # Check if width and height are valid
        if width <= 0 or height <= 0:
            print(f"Invalid ellipse dimensions: width={width}, height={height}")
            continue

        # Draw hollow ellipse (only lower half)
        cv2.ellipse(frame, (center_x, center_y), (width, height), 0, 0, 230, color, 2)  # Draw lower half with adjusted width

        # Add track ID
        cv2.putText(
            frame,
            f"#{obj.id}",
            (center_x - 10, center_y - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            color,
            2
        )

    video_writer.write(frame)

# Release resources
video_capture.release()
video_writer.release()
cv2.destroyAllWindows()
